In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.stats as ss

import sklearn.model_selection as ms 

In [2]:
# load your data (don't touch, just run)
"""
data = []
f = open('covtype.data','r')
while(1):
    line = f.readline()
    if  len(line) < 100:
        print line
    
    if len(line) == 0: break
    data.append(np.array([float(k) for k in line.split(',')]))
    if len(data) % 100000 == 0:
        print len(data)
        
f.close
data = np.vstack(data)
N = data.shape[0]
idx = np.random.permutation(N)


X_test = data[:N/5,:]
X_train = data[N/5:,:]
y_test = X_test[:,-1]
y_train = X_train[:,-1]
X_test = X_test[:,:-1]
X_train = X_train[:,:-1]


sio.savemat('covtype.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})

data = sio.loadmat('covtype.mat')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train'][0]
y_test = data['y_test'][0]

y_idx_train = [np.where(np.equal(y_train,k))[0] for k in np.unique(y_train)]

for i in xrange(len(y_idx_train)):
    y_idx = y_idx_train[i]
    y_idx_train[i] = y_idx[np.random.choice(len(y_idx),len(y_idx)/1000+1,replace=False)]
    
y_idx_train = np.hstack(y_idx_train)
y_idx_train = np.random.permutation(y_idx_train)

X_train = X_train[y_idx_train,:]
y_train = y_train[y_idx_train]

sio.savemat('covtype_reduced.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})


"""

"\ndata = []\nf = open('covtype.data','r')\nwhile(1):\n    line = f.readline()\n    if  len(line) < 100:\n        print line\n    \n    if len(line) == 0: break\n    data.append(np.array([float(k) for k in line.split(',')]))\n    if len(data) % 100000 == 0:\n        print len(data)\n        \nf.close\ndata = np.vstack(data)\nN = data.shape[0]\nidx = np.random.permutation(N)\n\n\nX_test = data[:N/5,:]\nX_train = data[N/5:,:]\ny_test = X_test[:,-1]\ny_train = X_train[:,-1]\nX_test = X_test[:,:-1]\nX_train = X_train[:,:-1]\n\n\nsio.savemat('covtype.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})\n\ndata = sio.loadmat('covtype.mat')\nX_train = data['X_train']\nX_test = data['X_test']\ny_train = data['y_train'][0]\ny_test = data['y_test'][0]\n\ny_idx_train = [np.where(np.equal(y_train,k))[0] for k in np.unique(y_train)]\n\nfor i in xrange(len(y_idx_train)):\n    y_idx = y_idx_train[i]\n    y_idx_train[i] = y_idx[np.random.choice(len(y_idx),len(y_idx)/1000+1,repla

In [3]:

data = sio.loadmat('covtype_reduced.mat')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train'][0]
y_test = data['y_test'][0]

print(np.unique(y_train), np.unique(y_test))

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


[1. 2. 3. 4. 5. 6. 7.] [1. 2. 3. 4. 5. 6. 7.]
(468, 54) (116202, 54) (468,) (116202,)


In [74]:
def pr_xlog2pr_x(pr_x):
    if(pr_x == 0):
        return 0
    return -1*pr_x*np.log2(pr_x)

def entropy_array(label):
    #     print(label)
    
    counts = np.unique(label, return_counts=True)[1]
#     print(counts)
    prob_x = counts / len(label)
#     print(prob_x)
    
    calc_pr_x_log2_pr_x = np.vectorize(pr_xlog2pr_x)
    
    pr_x_log2_pr_x = calc_pr_x_log2_pr_x(prob_x)
#     print(pr_x_log2_pr_x)
    return pr_x_log2_pr_x

def entropy(label):

    
    entropy = np.sum(entropy_array(label))

    return entropy

def pr_xylog2pr_x_given_y(pr_xy, pr_x_given_y):
    if(pr_x_given_y == 0):
        return 0
    return -1*pr_xy*np.log2(pr_x_given_y)

def cond_entropy_array(label, split):
    x_uniques = np.unique(label, return_counts=True)
    x_counts = x_uniques[1]
    prob_x = x_counts / len(label)
    
    y_uniques = np.unique(split, return_counts=True)
    y_counts = y_uniques[1]
    prob_y = y_counts / len(split)
    
    prob_x_and_y = np.zeros((len(x_uniques[0]),len(y_uniques[0])))
    prob_x_given_y = np.zeros((len(x_uniques[0]),len(y_uniques[0])))
    pr_xy_log2_pr_x_given = np.zeros((len(x_uniques[0]),len(y_uniques[0])))
    
    for x_index, x in enumerate(x_uniques[0]):
        get_all_x = (label == x)
        for y_index, y in enumerate(y_uniques[0]):
            get_all_y = (split == y)
            get_x_and_y = np.logical_and(get_all_x, get_all_y)
            x_and_y_counts = np.count_nonzero(get_x_and_y)
            prob_x_and_y[x_index][y_index] = x_and_y_counts / len(label)
            prob_x_given_y[x_index][y_index] = prob_x_and_y[x_index][y_index] / prob_y[y_index]
            pr_xy_log2_pr_x_given[x_index][y_index] = pr_xylog2pr_x_given_y(prob_x_and_y[x_index][y_index], prob_x_given_y[x_index][y_index])
    
#     print(prob_x_and_y)
#     print(prob_x_given_y)
#     print(pr_xy_log2_pr_x_given)
    
    return pr_xy_log2_pr_x_given

def cond_entropy(label,split):
    
    conditional_entropy = np.sum(cond_entropy_array(label, split))
    
    return conditional_entropy

random_sequences = sio.loadmat('random_sequences.mat')

s1 = random_sequences['s1'][0]
s2 = random_sequences['s2'][0]

print('entropy = ', entropy(s1))
print('conditional entropy = ', cond_entropy(s1,s2))

entropy =  3.3141823231610834
conditional entropy =  3.3029598816135173


In [235]:
def find_best_split(x,y):
#     print(x.T)
#     print(y)
    best_feat = 0
    splitval = 0.
    set1 = range(int(len(y)/2))
    set2 = range(int(len(y)/2),len(y))
    
#     max_ig = 0
#     best_feat = 0
#     split_val = 0
#     for index, feat in enumerate(x.T): 
# #         print(index)
#         h_y = entropy_array(y)
# #         print(h_y)
#         h_y_given_x = cond_entropy_array(y,feat)

#         x_uniques = np.unique(feat, return_counts=True)[0]

        
#         cond_entropies = np.sum(h_y_given_x, axis=1)
# #         print(cond_entropies)
#         igs = h_y - cond_entropies
# #         print(igs)
        
#         max_ig_local = max(igs)
#         max_ig_local_index = np.argmax(igs)
# #         print(max_ig_local, max_ig_local_index)
#         if(max_ig_local > max_ig):
#             max_ig = max_ig_local
#             best_feat = index
#     var = x.T[best_feat]
#     var = set(var)
    
    
#     ordered_uniques = sorted(var)
# #     print(ordered_uniques)
#     max_ig = 0
#     for i in range(0, int(max(x.T[best_feat])), 10):
#         indices = np.where(x.T[best_feat]>i)
#         if(len(y[indices]) > 0):
#             h_y = entropy_array(y[indices])
#             h_y_given_x = cond_entropy_array(y[indices], x.T[best_feat, indices])
#     #         print(h_y)
#     #         print(h_y_given_x)
#             h_y_given_x = np.sum(h_y_given_x,axis=1)
#             ig = h_y-h_y_given_x
#     #         print(ig)
#             max_ig_local = np.argmax(ig)
#             if(max_ig_local > max_ig):
#                 max_ig = max_ig_local
#                 splitval = i
        
        
        
    max_ig = 0
    best_feat = 0
    splitval = 1
    print(x.shape)
    for index, f in enumerate(x.T):
        for val in range(max(1,int(min(f))), int(max(f))):
            
            split = f > val
            split = np.where(split)[0]
            split2 = f<=val
            split2 = np.where(split2)[0]
            h_s = entropy(y)
            h_sl = entropy(y[split])
            h_sr = entropy(y[split2])
            h_s_given_split = ((len(split)/len(y))*h_sl) + ((len(split2)/len(y))*h_sr)
            ig = h_s - h_s_given_split
            if(ig>max_ig):
                max_ig = ig
                best_feat = index
                splitval = val
        
        
        
#     print(splitval)
#         print()
#         local_max_ig = 0
#         local_max_ig_x = 0
#         for y_index in range(len(h_y_given_x)):
#             for x_index in range(len(h_y_given_x[0])):
#                 ig = h_y[y_index] - h_y_given_x[y_index][x_index]
                
#                 if(ig > local_max_ig):
#                     local_max_ig = max(local_max_ig, ig)
#                     local_max_ig_x = x_uniques[x_index]
#         print(min(feat))
                
#         if(local_max_ig > max_ig):
#             max_ig = local_max_ig
#             best_feat = index
#             splitval = local_max_ig_x
#     print(best_feat, x.T[best_feat])
            
#     print(best_feat)
#     print(x[:, best_feat])
#     print(splitval)
    arr = x[:,best_feat]
#     print(arr)
    set1 = arr > splitval
    set1 = np.where(set1)[0]
    
    set2 = arr <= splitval
    set2 = np.where(set2)[0]
#     print(set1)
    return best_feat, splitval, set1, set2


best_feat, splitval, set1, set2 = find_best_split(X_train, y_train)
print("split feature:", best_feat)
print("split value:", splitval)
y_new = y_train * 0
y_new[set1] = 1
print('information gained in first step', entropy(y_train) - cond_entropy(y_train,y_new))

(468, 54)
split feature: 0
split value: 2843
information gained in first step 0.34265604135138417


In [256]:
def purity(y): #split next node based on most purity
    return ss.mode(y)[1]/len(y+0.)
    


class Node:
    def __init__(self,  sample_idx, nodeid,  is_leaf = True):
        self.is_leaf = is_leaf
        self.id = nodeid
        self.sample_idx = sample_idx
        self.children = []
        
        
        
    def visit_node(self, x):
        if self.is_leaf:
            return self.label
        """ Fill me in (x is list of features, check node and see feature and split value to decide to go to left or right node) """ 
        if(x[self.splitfeat] > splitval):
        #         print(x)
            return self.children[0].visit_node(x)
        else: 
            return self.children[1].visit_node(x)
        
    def add_split_details(self, splitfeat, splitval)  :
        self.splitfeat = splitfeat
        self.splitval = splitval
    
        
class Tree:
    def __init__(self, x,y):
        m = len(y)
        self.x = x
        self.y = y
        self.maxid = -1
        self.root = self.construct_node(np.array(range(m)))
        self.leaves = [self.root]
        
    def print_tree(self):
        print('printing tree...')
        def print_node(parent, node):
            print(node.id, end='')
            
            if parent is not None:
                print(', parent ', parent.id,end='')
            else:
                print(', ROOT',end='')
                
            print(', label ', node.label, end='')
            if node.is_leaf: 
                print(', LEAF, ', 'nsamples %d, purity %.2f' %(len(node.sample_idx), purity(self.y[node.sample_idx])))
            else:
                print(', NONLEAF, split %d, val %.2f' % (node.splitfeat, node.splitval))
            if not node.is_leaf:
                for ch in node.children:
                    print_node(node, ch)
        print_node(None, self.root)
        
        
    def construct_node(self, sample_idx):
#         print(self.y[sample_idx])
        node = Node(sample_idx, self.maxid + 1,  True)
        node.label = round(np.mean(self.y[sample_idx]))#fill me in
        node.entropy = entropy(self.y[sample_idx])
        node.num_mistakes = np.sum(np.not_equal(node.label, self.y[sample_idx]))
        self.maxid += 1
        return node
        


    def report_train_err(self):
        total_mistakes = 0
        for leaf in self.leaves:
            total_mistakes += leaf.num_mistakes
        return total_mistakes / (len(self.y)+0.)
        
    
    
    def predict(self,x):
        return self.root.visit_node(x)


    

In [260]:
def get_test_err(tree):
    # get test error
    num_test_mistakes = 0
    for k in range(len(y_test)):
#     for k in range(1):
        x,y = X_test[k,:],y_test[k]
#         print(x)
        if y != tree.predict(x):
            num_test_mistakes += 1
    return num_test_mistakes / (len(y_test)+0.)



tree = Tree(X_train,y_train)
tree.print_tree()
print('current train err:', tree.report_train_err())
print('current test err:', get_test_err(tree))


# my first split
best_feat, splitval, set1, set2 = find_best_split(X_train, y_train)
 
left_child = tree.construct_node(set1)
right_child = tree.construct_node(set2)
tree.root.is_leaf = False
tree.leaves.pop(tree.leaves.index(tree.root))
tree.root.add_split_details(splitfeat = best_feat, splitval = splitval)


tree.root.children = [left_child, right_child]
tree.leaves.extend(tree.root.children)
tree.print_tree()
print('one step train err:', tree.report_train_err())
print('one step test err:', get_test_err(tree))

for i in range(24):
# while True:
    min_purity = float('inf')
    best_leaf= tree.leaves[0]
#     print(tree.y)
    
    for leaf in tree.leaves:
#         print(leaf.sample_idx)
        purityval = purity(tree.y[leaf.sample_idx])
#         print(min_purity)

        if(purityval < min_purity ):
            min_purity = purityval
            best_leaf = leaf
    if(min_purity == 1):
        break
    print("splitting node:", best_leaf.id)
    best_feat, splitval, set1, set2 = find_best_split(tree.x[best_leaf.sample_idx], y_train[best_leaf.sample_idx])
    left_child = tree.construct_node(set1)
    right_child = tree.construct_node(set2)
    best_leaf.is_leaf = False
    tree.leaves.pop(tree.leaves.index(best_leaf))
#     print(tree.leaves)
    best_leaf.add_split_details(splitfeat = best_feat, splitval = splitval)
    
    best_leaf.children = [left_child, right_child]
    tree.leaves.extend(best_leaf.children)
    string=""
    for leaf in tree.leaves:
        string += str(leaf.id) +", "
    print(i,'step train err:', tree.report_train_err())
    print(i, 'step test err:', get_test_err(tree))
#     print(string)
tree.print_tree()
print('25 step train err:', tree.report_train_err())
print('25 step test err:', get_test_err(tree))

printing tree...
0, ROOT, label  2, LEAF,  nsamples 468, purity 0.44
current train err: 0.5641025641025641
current test err: 0.3138069912738163
(468, 54)
printing tree...
0, ROOT, label  2, NONLEAF, split 0, val 2843.00
1, parent  0, label  1, LEAF,  nsamples 352, purity 0.51
2, parent  0, label  2, LEAF,  nsamples 116, purity 0.46
one step train err: 0.5021367521367521
one step test err: 0.4959983477048588
splitting node: 2
(116, 54)
0 step train err: 0.49145299145299143
0 step test err: 0.7755288205022289
splitting node: 3
(79, 54)
1 step train err: 0.4829059829059829
1 step test err: 0.38671451438013116
splitting node: 6
(21, 54)
2 step train err: 0.47649572649572647
2 step test err: 0.7846250494827972
splitting node: 1
(352, 54)
3 step train err: 0.5341880341880342
3 step test err: 0.3138069912738163
splitting node: 10
(220, 54)
4 step train err: 0.5149572649572649
4 step test err: 0.3138069912738163
splitting node: 9
(132, 54)
5 step train err: 0.5170940170940171
5 step test err: 